In [1]:
from pgmpy.models import BayesianNetwork
from pgmpy.factors.discrete import TabularCPD
from pgmpy.sampling import BayesianModelSampling
from pgmpy.inference import VariableElimination

In [2]:
# Define the Bayesian Network
fraudNet = BayesianNetwork([("T", "F"), ("T", "P"), ("F", "P"), ("F", "I"), ("C", "I"), ("C", "A")])

# Define the CPDs
cpd_T = TabularCPD('T', 2, [[0.92], [0.08]])
cpd_C = TabularCPD('C', 2, [[0.40], [0.60]])

cpd_F = TabularCPD('F', 2, [[0.998, 0.98], [0.002, 0.02]], ["T"], [2])
cpd_A = TabularCPD('A', 2, [[0.999, 0.9], [0.001, 0.1]], ["C"], [2])

cpd_P = TabularCPD('P', 2, [[0.99, 0.9, 0.1, 0.1], [0.01, 0.1, 0.9, 0.9]], ['T', 'F'], [2, 2])
cpd_I = TabularCPD('I', 2, [[0.999, 0.99, 0.989, 0.98], [0.001, 0.01, 0.011, 0.02]], ['F', 'C'], [2, 2])

# Add CPDs to the model
fraudNet.add_cpds(cpd_T, cpd_C, cpd_F, cpd_A, cpd_P, cpd_I)

C:\Users\bhara\AppData\Local\Temp\ipykernel_4652\3848405760.py:8: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  cpd_F = TabularCPD('F', 2, [[0.998, 0.98], [0.002, 0.02]], ["T"], [2])
C:\Users\bhara\AppData\Local\Temp\ipykernel_4652\3848405760.py:9: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  cpd_A = TabularCPD('A', 2, [[0.999, 0.9], [0.001, 0.1]], ["C"], [2])
C:\Users\bhara\AppData\Local\Temp\ipykernel_4652\3848405760.py:11: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  cpd_P = TabularCPD('P', 2, [[0.99, 0.9, 0.1, 0.1], [0.01, 0.1, 0.9, 0.9]], ['T', 'F'], [2, 2])
C:\Users\bhara\AppData\Local\Temp\ipykernel_4652\3848405760.py:12: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` in

In [3]:
print(fraudNet.get_cpds()[5])

+------+-------+------+-------+------+
| F    | F(0)  | F(0) | F(1)  | F(1) |
+------+-------+------+-------+------+
| C    | C(0)  | C(1) | C(0)  | C(1) |
+------+-------+------+-------+------+
| I(0) | 0.999 | 0.99 | 0.989 | 0.98 |
+------+-------+------+-------+------+
| I(1) | 0.001 | 0.01 | 0.011 | 0.02 |
+------+-------+------+-------+------+


In [4]:
# Prior Probability of Fraud
samples = BayesianModelSampling(fraudNet).likelihood_weighted_sample(evidence = [], size = 100000, seed = 123)
print(samples["F"].value_counts(normalize = True))

inference = VariableElimination(fraudNet)
print(inference.query(['F'], evidence = {}))

  0%|          | 0/6 [00:00<?, ?it/s]

0    0.99667
1    0.00333
Name: F, dtype: float64
+------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9966 |
+------+----------+
| F(1) |   0.0034 |
+------+----------+


In [5]:
samples = BayesianModelSampling(fraudNet).likelihood_weighted_sample(evidence = [("P", 0), ("I", 1), ("A", 0)], size = 100000, seed = 123)
total_weight = samples._weight.sum()
print(samples[samples["F"] == 1]["_weight"].sum()/total_weight)

inference = VariableElimination(fraudNet)
print(inference.query(['F'], evidence = {"P":0, "I":1, "A":0}))

  0%|          | 0/6 [00:00<?, ?it/s]

0.0051847827934045415
+------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9948 |
+------+----------+
| F(1) |   0.0052 |
+------+----------+


In [6]:
samples = BayesianModelSampling(fraudNet).likelihood_weighted_sample(evidence = [("P", 0), ("I", 1), ("A", 0), ("T", 1)], size = 100000, seed = 123)
total_weight = samples._weight.sum()
print(samples[samples["F"] == 1]["_weight"].sum()/total_weight)

inference = VariableElimination(fraudNet)
print(inference.query(['F'], evidence = {"P":0, "I":1, "A":0, "T":1}))

  0%|          | 0/6 [00:00<?, ?it/s]

0.05113310542253096
+------+----------+
| F    |   phi(F) |
+======+==========+
| F(0) |   0.9492 |
+------+----------+
| F(1) |   0.0508 |
+------+----------+
